# Categorical Variables
Antes de apresentar qualquer técnica, é importante destacar que __é imprescindível o entendimento do negócio e da base de dados__. Apenas desta forma será possível escolher a melhor opção para trabalhar com os dados.

Este notebook apresenta 3 abordagens para lidar com variáveis categóricas. 
1. [Exclusão de colunas com Variáveis Categóricas.](#approach1)
2. [Inclusão de valores através da técnica LabelEncoder.](#approach2)
3. [Inclusão de valores através da técnica OneHotEncoder.](#approach3)

In [1]:
import pandas as pd

df = pd.read_csv('data.csv')
df

,Address,Price
0,13 Robinson St,1700000.0
1,2 Bruce St,1500000.0
2,2 Bruce St,2000000.0
3,2 Bruce St,2300000.0
4,176 Barkly St,1925000.0
5,13 Robinson St,637000.0
6,13 Robinson St,559000.0


## Seleção de features

In [2]:
# Obtém uma lista com os nomes das colunas que contém variáveis categóricas
categorical_cols = df.select_dtypes(include='object').columns.tolist()
categorical_cols

['Address']

<a id="approach1"></a>

## Abordagem #1: Remover colunas do tipo Categorical
Esta técnica deve ser evitada devida à perca de dados. Portanto, certifique-se de conhecer o negócio e a base de dados.

In [3]:
# Removemos todas as colunas do tipo object, ou seja, mantemos apenas as numéricas
df1 = df.drop(columns=categorical_cols)
df1.head()

,Price
0,1700000.0
1,1500000.0
2,2000000.0
3,2300000.0
4,1925000.0


<a id="approach2"></a>

## Abordagem #2: Label Encoding
Scikit-learn possui a classe LabelEncoder que pode ser utilizada para converter variáveis categóricas em numéricas. Portanto, criaremos uma estrutura de repetição nas variáveis categóricas e aplicaremos o LabelEncoder separadamente nas colunas.

Doc: [sklearn.preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

__Dica__: Caso você trabalhe com Pipelines, utilize [sklearn.preprocessing.OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)

In [4]:
# Cria uma cópia do DataFrame para que o original não seja modificado
df2 = df.copy()

In [5]:
from sklearn.preprocessing import LabelEncoder

# Aplicamos o LabelEncoder em cada coluna com Categorical Variables
encoder = LabelEncoder()
for col in categorical_cols:
    df2[col] = encoder.fit_transform(df2[col])

__Compare as colunas com dados categóricos apresentados nas linhas seguintes__

#### Base de dados original

In [6]:
# Base de dados original
df[categorical_cols].head(10)

,Address
0,13 Robinson St
1,2 Bruce St
2,2 Bruce St
3,2 Bruce St
4,176 Barkly St
5,13 Robinson St
6,13 Robinson St


#### Base de dados modificada com o LabelEncoder
Observe que:
* "13 Robinson St" corresponde ao valor 0
* "176 Barkly St" corresponde ao valor 1
* "2 Bruce St" corresponde ao valor 2

In [7]:
# Base de dados modificada com o LabelEncoder 
df2[categorical_cols].head(10)

,Address
0,0
1,2
2,2
3,2
4,1
5,0
6,0


<a id="approach3"></a>

## Abordagem #3: One-Hot Encoding
One-Hot Encoding é responsável por converter variáveis categóricas em colunas.

Na coluna "Address" possuímos 3 valores:
1. 13 Robinson St
2. 2 Bruce St
3. 176 Barkly St

Para cada valor será criada uma coluna que receberá os valores 0 (False) ou 1 (True).

Doc: [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [8]:
# Cria uma cópia do DataFrame para que o original não seja modificado
df3 = df.copy()

In [9]:
from sklearn.preprocessing import OneHotEncoder

# Nós configuramos o parâmetro sparse=False para garantir que o retorno do método seja um array.
ohencoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Aplicamos o One-Hot Encoding para todas as variáveis categóricas
oh_df3 = pd.DataFrame(ohencoder.fit_transform(df3[categorical_cols]))
oh_df3

,0,1,2
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,1.0,0.0
5,1.0,0.0,0.0
6,1.0,0.0,0.0


__Unimos o resultado de One-Hot Enconding com o DataFrame original__

Observe que:
* "13 Robinson St" corresponde à coluna 0
* "176 Barkly St" corresponde à coluna 1
* "2 Bruce St" corresponde à coluna 2

In [10]:
df3 = df3.merge(oh_df3,  left_index=True, right_index=True)
df3

,Address,Price,0,1,2
0,13 Robinson St,1700000.0,1.0,0.0,0.0
1,2 Bruce St,1500000.0,0.0,0.0,1.0
2,2 Bruce St,2000000.0,0.0,0.0,1.0
3,2 Bruce St,2300000.0,0.0,0.0,1.0
4,176 Barkly St,1925000.0,0.0,1.0,0.0
5,13 Robinson St,637000.0,1.0,0.0,0.0
6,13 Robinson St,559000.0,1.0,0.0,0.0
